In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from selenium.webdriver.common.by import By

url='https://www.premierleague.com/'

# get the webpage
webpage = requests.get(url)

# load webpage into bs4
soup = BeautifulSoup(webpage.content, 'html.parser')

In [ ]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get(url)

In [ ]:
time.sleep(6) #wait for page to load

driver.find_element(By.XPATH,"/html/body/div[2]/div/div/div[1]/div[5]/button[1]").click() # accept cookies
time.sleep(2)

driver.find_element(By.XPATH,"/html/body/header/div/nav/ul/li[10]/a").click() 
time.sleep(6)

player_name =[]
player_position = []
player_nationality = []
player_club = []
player_season = []
player = []

soup = BeautifulSoup(driver.page_source, 'html.parser') #to rerun after change page

seasonSeq = 2

while seasonSeq <= 4: # previous 3 seasons
    
    driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div/section/div[1]/div[2]").click() # open Season
    time.sleep(1)
    driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div/section/div[1]/ul/li[" + str(seasonSeq) + "]").click() # click from 2020/21
    time.sleep(3)
    
    clubSeq = 2 # reset counter
    
    while clubSeq <= 21:
        
        driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div/section/div[2]/div[2]").click() #open Club
        time.sleep(1)
        driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div/section/div[2]/ul/li[" + str(clubSeq) + "]").click() # click from first Club, total 20 Club every year
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser') #to rerun after change page

        pName_list = soup.find('tbody', attrs={'class':'dataContainer indexSection'}).find_all('a', attrs={'class':'playerName'})
        for pName_items in pName_list:
            player_name.append(pName_items.text)
            current_club = soup.find('div', attrs={'data-dropdown-current':'clubs'})
            player_club.append(current_club.text)
            current_playerSeason = soup.find('div', attrs={'data-dropdown-current':'compSeasons'})
            player_season.append(current_playerSeason.text)

        pPosition_list = soup.find('tbody', attrs={'class':'dataContainer indexSection'}).find_all('tr')

        for pPosition_items in pPosition_list:
            for pPosition_items2 in pPosition_items.find('td',attrs={'class':'hide-s'}):
                player_position.append(pPosition_items2.text)

        pCountry_list = soup.find('tbody', attrs={'class':'dataContainer indexSection'}).find_all('span', attrs={'class':'playerCountry'})

        for pCountry_items in pCountry_list:
            player_nationality.append(pCountry_items.text)

        clubSeq = clubSeq + 1
        
        
    seasonSeq = seasonSeq + 1


In [ ]:
player = {'Season': player_season,
          'Club': player_club,
         'Name':player_name,
         'Position':player_position,
         'Nationatility':player_nationality}


In [ ]:
pl_player_df = pd.DataFrame(player)
pl_player_df

In [ ]:
pl_player_df.sort_values(by=['Season','Club','Position'])
pl_player_df.to_csv('premierleague_player.csv', index=False)

In [ ]:
##driver.close()